In [ ]:
import ants
import os


In [2]:
main_path = "/home/jovyan/work/data/DATASET/"
selected_dataset = ["OASIS3","AMY-MR","AMYFDGMR-MOV","Ginevra","MRADV-PT","FDGMR-FTD"]

data = []
for dataset in selected_dataset:
    dataset_path = os.path.join(main_path, dataset)
    if not os.path.isdir(dataset_path):
        print(f"Warning: Il dataset {dataset} non esiste nel percorso {dataset_path}.")
        continue

    try:
        subjects = os.listdir(dataset_path)
    except Exception as e:
        print(f"Errore durante l'accesso ai soggetti nel dataset {dataset}: {e}")
        continue

    for subject in subjects:
        
        mr_dir = os.path.join(dataset_path,subject)
        ct_dir = os.path.join(dataset_path, subject, "processed", "CT", "CTonMR")

        try:
            mr_image_path = next(
                os.path.join(mr_dir, image)
                for image in os.listdir(mr_dir)
                if "t1" in image
            )
        except (StopIteration, FileNotFoundError):
            print(f"Warning: File MR mancante per il soggetto {subject} nel dataset {dataset}.")
            mr_image_path = None

        try:
            ct_image_path = next(
                os.path.join(ct_dir, image)
                for image in os.listdir(ct_dir)
                if "ctMRspace" in image
            )
        except (StopIteration, FileNotFoundError):
            print(f"Warning: File CT mancante per il soggetto {subject} nel dataset {dataset}.")
            ct_image_path = None
        if mr_image_path and ct_image_path:
                data.append({
                    "CT": ct_image_path,
                    "MR": mr_image_path,
                    "subject": subject,
                    "dataset": dataset
                })
        else:
                missing_modalities = []
                if not mr_image_path:
                    missing_modalities.append("MR")
                if not ct_image_path:
                    missing_modalities.append("CT")
                print(f"Warning: Modalità mancanti per il soggetto {subject} nel dataset {dataset}: {', '.join(missing_modalities)}")



In [3]:
test_images = data[0]

In [6]:

def processing(data):
    """
    Esegue la registrazione di immagini CT e MR a un template MNI e salva i risultati.

    Args:
        data (dict): Un dizionario contenente i percorsi delle immagini CT e MR.
                     Es: {"CT": "/path/to/ct.nii.gz", "MR": "/path/to/mr.nii.gz"}
    """
    # Caricamento delle immagini
    try:
        ct = ants.image_read(data["CT"])
        mr = ants.image_read(data["MR"])
        template = ants.image_read("/home/jovyan/work/data/mni_152_stripped.nii.gz")
    except Exception as e:
        print(f"Errore durante il caricamento delle immagini: {e}")
        return

    print("Immagini caricate con successo.")

    try:
        mrotontemplate = ants.registration(mr, template, "antsRegistrationSyNQuick[a]")
        print("Registrazione MR al template completata.")
    except Exception as e:
        print(f"Errore durante la registrazione MR: {e}")
        return

    try:
        ctontemplate = ants.apply_transforms(fixed=template,
                                            moving=ct,
                                            transformlist=mrotontemplate['fwdtransforms'], # Corretto: usa le trasformazioni da mrotontemplate
                                            interpolator='bSpline')
        print("Trasformazione CT al template completata.")
    except Exception as e:
        print(f"Errore durante la trasformazione CT: {e}")
        return

    ct_output_dir = os.path.dirname(data["CT"]).replace("CTonMR", "CTonTemplate")
    ct_output_filename = os.path.basename(data["CT"]).replace("ctMRspace", "ctTemplateSpaceNoskull")
    ct_output = os.path.join(ct_output_dir, ct_output_filename)

    mr_output_dir = os.path.dirname(data["MR"]).replace("CTonMR", "CTonTemplate")
    mr_output_filename = os.path.basename(data["MR"]).replace("_t1w", "-mrTemplateSpaceNoskull")
    mr_output = os.path.join(mr_output_dir, mr_output_filename)

    mr_output_norm_filename = os.path.basename(data["MR"]).replace("_t1w", "-mrTemplateSpaceNoskullNormalized")
    mr_output_norm = os.path.join(mr_output_dir, mr_output_norm_filename)

    # --- Salvataggio delle immagini ---
    try:

        ants.image_write(ctontemplate, ct_output)
        print(f"Immagine CT salvata in: {ct_output}")

    except Exception as e:
        print(f"Errore durante il salvataggio delle immagini: {e}")

    try :
        ants.image_write(mrotontemplate['warpedmovout'], mr_output)
        print(f"Immagine MR salvata in: {mr_output}")


    except Exception as e:
        print(f"Errore durante il salvataggio delle immagini: {e}")

    print("Processo di elaborazione completato.")

    
    

In [14]:
import ants
import os

def processing(data):
    ct = ants.image_read(data["CT"])
    mr = ants.image_read(data["MR"])
    template = ants.image_read("/home/jovyan/work/data/mni_icbm152_t1_tal_nlin_asym_09c.nii")

    mrotontemplate = ants.registration(fixed=template, moving=mr, type_of_transform="antsRegistrationSyNQuick[a]")

    ctontemplate = ants.apply_transforms(fixed=template,
                                        moving=ct,
                                        transformlist=mrotontemplate['fwdtransforms'],
                                        interpolator='bSpline')

    ct_output_dir = os.path.dirname(data["CT"]).replace("DATASET", "mr2ct").replace("CTonMR", "CTonTemplate")
    ct_output_filename = os.path.basename(data["CT"]).replace("ctMRspace", "ctTemplateSpaceNoskull")
    ct_output = os.path.join(ct_output_dir, ct_output_filename)

    os.makedirs(ct_output_dir, exist_ok=True)

    mr_output_dir = os.path.dirname(data["MR"]).replace("DATASET", "mr2ct").replace("CTonMR", "CTonTemplate")
    # CORREZIONE: sintassi di os.path.join
    mr_output_filename = os.path.basename(data["MR"]).replace("_t1w", "-mrTemplateSpaceNoskull")
    mr_output = os.path.join(mr_output_dir, "processed", "MR", "MRonTemplate", mr_output_filename)


    os.makedirs(os.path.dirname(mr_output), exist_ok=True) # Crea la directory completa per mr_output

    mr_output_norm_filename = os.path.basename(data["MR"]).replace("_t1w", "-mrTemplateSpaceNoskullNormalized")
    mr_output_norm = os.path.join(mr_output_dir, mr_output_norm_filename)

    try:
        ants.image_write(ctontemplate, ct_output)
        ants.image_write(mrotontemplate['warpedmovout'], mr_output)
    except Exception as e:
        print(f"Errore durante il salvataggio delle immagini: {e}")

In [15]:
import ants
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

def processing(data):
    """
    Processa una singola coppia di immagini CT e MR, registrandole a un template
    e salvando i risultati in percorsi specifici.

    Args:
        data (dict): Un dizionario contenente i percorsi delle immagini CT e MR.
                     Es: {"CT": "/path/to/ct.nii.gz", "MR": "/path/to/mr.nii.gz"}
    Returns:
        str: Un messaggio di successo o errore per l'elemento processato.
    """
    try:
        ct = ants.image_read(data["CT"])
        mr = ants.image_read(data["MR"])
        template = ants.image_read("/home/jovyan/work/data/mni_152_stripped.nii.gz")

        # Registrazione del MR al template (fixed = template, moving = mr)
        mrotontemplate = ants.registration(fixed=template, moving=mr, type_of_transform="antsRegistrationSyNQuick[a]")

        # Applicazione della trasformazione CT al template
        ctontemplate = ants.apply_transforms(fixed=template,
                                            moving=ct,
                                            transformlist=mrotontemplate['fwdtransforms'],
                                            interpolator='bSpline')

        # --- Creazione e gestione dei percorsi di output ---
        ct_output_dir = os.path.dirname(data["CT"]).replace("DATASET", "mr2ct").replace("CTonMR", "CTonTemplate")
        ct_output_filename = os.path.basename(data["CT"]).replace("ctMRspace", "ctTemplateSpaceNoskull")
        ct_output = os.path.join(ct_output_dir, ct_output_filename)

        os.makedirs(ct_output_dir, exist_ok=True)

        mr_output_dir = os.path.dirname(data["MR"]).replace("DATASET", "mr2ct").replace("CTonMR", "CTonTemplate")
        mr_output_filename = os.path.basename(data["MR"]).replace("_t1w", "-mrTemplateSpaceNoskull")
        # Attenzione: os.path.join accetta argomenti multipli per le sottocartelle
        mr_output = os.path.join(mr_output_dir, "processed", "MR", "MRonTemplate", mr_output_filename)

        # Crea la directory completa per mr_output
        os.makedirs(os.path.dirname(mr_output), exist_ok=True)

        # Il percorso mr_output_norm_filename non è utilizzato per il salvataggio in questa funzione
        mr_output_norm_filename = os.path.basename(data["MR"]).replace("_t1w", "-mrTemplateSpaceNoskullNormalized")
        mr_output_norm = os.path.join(mr_output_dir, mr_output_norm_filename)

        # --- Salvataggio delle immagini ---
        ants.image_write(ctontemplate, ct_output)
        ants.image_write(mrotontemplate['warpedmovout'], mr_output)

        return f"Processato con successo CT: {data['CT']} e MR: {data['MR']}"

    except Exception as e:
        return f"Errore durante l'elaborazione di {data.get('CT', 'N/A')} e {data.get('MR', 'N/A')}: {e}"

---

def parallel_processing(list_of_data_dicts, max_workers=None):
    """
    Parallelizza l'esecuzione della funzione 'processing' su una lista di dizionari.

    Args:
        list_of_data_dicts (list): Una lista di dizionari, dove ogni dizionario
                                   contiene i percorsi "CT" e "MR" per una coppia di immagini.
        max_workers (int, optional): Il numero massimo di processi da utilizzare.
                                     Se None, userà il numero di CPU del sistema.
    """
    print(f"Avvio elaborazione parallela su {len(list_of_data_dicts)} elementi con {max_workers if max_workers else 'numero predefinito di CPU'} worker.")
    # Utilizziamo ProcessPoolExecutor per operazioni CPU-bound
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Invia tutte le task al pool di processi
        future_to_data = {executor.submit(processing, data_item): data_item for data_item in list_of_data_dicts}

        # Iteriamo sui risultati man mano che diventano disponibili
        for future in as_completed(future_to_data):
            data_item = future_to_data[future]
            try:
                result = future.result()
                print(result) # Stampa il risultato (successo/errore) di ogni task
            except Exception as exc:
                print(f'{data_item} ha generato un\'eccezione: {exc}')

    print("Elaborazione parallela completata.")

# --- Esempio di utilizzo ---
if __name__ == "__main__":
    parallel_processing(sample_data_list, max_workers=4)